In [1]:
import os
import pickle
from PIL import Image
import numpy as np
import glob
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.links.caffe import CaffeFunction
from chainer.datasets import LabeledImageDataset

from matplotlib import pylab as plt

#  データセットを作る

In [2]:
from itertools import chain
# 画像フォルダ
IMG_DIR = 'datas'
# 各注視点ごとのフォルダ
dnames = glob.glob('{}/*'.format(IMG_DIR))
# キャプチャのパス
fnames = [glob.glob('{}/*.jpg'.format(d)) for d in dnames
          if not os.path.exists('{}/ignore'.format(d))]
fnames = list(chain.from_iterable(fnames))

labels = [os.path.basename(os.path.dirname(fn)) for fn in fnames]
dnames = [os.path.basename(d) for d in dnames
          if not os.path.exists('{}/ignore'.format(d))]
labels = [dnames.index(l) for l in labels]

d = LabeledImageDataset(list(zip(fnames, labels)))

# データセットを加工する

In [3]:
from chainer.datasets import TransformDataset
from PIL import Image

width, height = 224, 224

# 平均画像を用意する
mean_image = np.ndarray((3, 224, 224), dtype=np.float32)
mean_image[0] = 103.939
mean_image[1] = 116.779
mean_image[2] = 123.68

# 各データに行う変換
def transform(inputs):
    img, label = inputs
    img = img[:3, ...]
    img = img.astype(np.uint8)
    img = Image.fromarray(img.transpose(1, 2, 0))
    img = img.resize((width, height), Image.BICUBIC)
    img = np.asarray(img).transpose(2, 0, 1)
    img = img - mean_image
    return img, label

# 変換付きデータセットにする
td = TransformDataset(d, transform)

In [4]:
from chainer import datasets

train_img, valid_img = datasets.split_dataset_random(td, int(len(d) * 0.8), seed=0)

# モデルの読み込み

In [5]:
%%time
MODEL = 'model/bvlc_googlenet.caffemodel'
PICKLE = 'model/ggnet.pkl'
if os.path.exists(PICKLE):
    print("Load pickle")
    with open(PICKLE, 'rb') as pkl:
        model = pickle.load(pkl)
else:
    print("Load caffemodel and make pickle")
    if os.path.exists(MODEL):
        model = CaffeFunction(MODEL)
        with open(PICKLE, 'wb') as pkl:
            pickle.dump(model, pkl)
    print(MODEL + " not found.")

Load caffemodel and make pickle
model/bvlc_googlenet.caffemodel not found.
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 525 µs


# GoogLeNetのモデル

In [6]:
class GoogLeNet(Chain):
    insize = 224
    
    def __init__(self):
        super(GoogLeNet, self).__init__()
        with self.init_scope():
            self.conv1 =  L.Convolution2D(3, 64, 7, stride=2, pad=3)
            self.conv2_reduce = L.Convolution2D(64, 64, 1)
            self.conv2 = L.Convolution2D(64, 192, 3, stride=1, pad=1)
            self.inception_3a = L.Inception(192, 64, 96, 128, 16, 32, 32)
            self.inception_3b = L.Inception(256, 128, 128, 192, 32, 96, 64)
            self.inception_4a = L.Inception(480, 192, 96, 208, 16, 48, 64)
            self.inception_4b = L.Inception(512, 160, 112, 224, 24, 64, 64)
            self.inception_4c = L.Inception(512, 128, 128, 256, 24, 64, 64)
            self.inception_4d = L.Inception(512, 112, 144, 288, 32, 64, 64)
            self.inception_4e = L.Inception(528, 256, 160, 320, 32, 128, 128)
            self.inception_5a = L.Inception(832, 256, 160, 320, 32, 128, 128)
            self.inception_5b = L.Inception(832, 384, 192, 384, 48, 128, 128)
            self.loss3_fc = L.Linear(1024, 1000)
            
            self.loss1_conv = L.Convolution2D(512, 128, 1)
            self.loss1_fc1 = L.Linear(2048, 1024)
            self.loss1_fc2 = L.Linear(1024, 1000)
            
            self.loss2_conv = L.Convolution2D(528, 128, 1)
            self.loss2_fc1 = L.Linear(2048, 1024)
            self.loss2_fc2 = L.Linear(1024, 1000)
        
    def __call__(self, x):
        h = F.relu(self.conv1(x))
        h = F.max_pooling_2d(h, 3, stride=2)
        h = F.local_response_normalization(h, n=5, k=1, alpha=2e-05)
        h = F.relu(self.conv2_reduce(h))
        h = F.relu(self.conv2(h))
        h = F.local_response_normalization(h, n=5, k=1, alpha=2e-05)
        h = F.max_pooling_2d(h, 3, stride=2)
        
        h = self.inception_3a(h)
        h = self.inception_3b(h)
        h = F.max_pooling_2d(h, 3, stride=2)
        h = self.inception_4a(h)
        
        b = F.average_pooling_2d(h, 5, stride=3)
        b = F.relu(self.loss1_conv(b))
        b = F.relu(self.loss1_fc1(b))
        b = self.loss1_fc2(b)
        
        h = self.inception_4b(h)
        h = self.inception_4c(h)
        h = self.inception_4d(h)
        
        b = F.average_pooling_2d(h, 5, stride=3)
        b = F.relu(self.loss2_conv(b))
        b = F.relu(self.loss2_fc1(b))
        b = self.loss2_fc2(b)
        
        h = self.inception_4e(h)
        h = F.max_pooling_2d(h, 3, stride=2)
        h = self.inception_5a(h)
        h = self.inception_5b(h)
        
        h = F.average_pooling_2d(h, 7, stride=1)
        y = self.loss3_fc(F.dropout(h, 0.4))
        return y

In [9]:
def train(train, test):
    model = L.Classifier(GoogLeNet())
    dev = 0
    if dev >= 0:
        chainer.cuda.get_device(dev).use()
        model.to_gpu()

    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)

    train_iter = chainer.iterators.SerialIterator(train, 50)
    test_iter = chainer.iterators.SerialIterator(test, 50,repeat=False, shuffle=False)

    epoch = 5000
    updater = training.StandardUpdater(train_iter, optimizer, device=dev)
    trainer = training.Trainer(updater, (epoch, 'epoch'), out="result")
    
    # Evaluator
    trainer.extend(extensions.Evaluator(test_iter, model, device=dev))

    # LogReport
    trainer.extend(extensions.LogReport())

    # PrintReport
    trainer.extend(extensions.PrintReport( entries=['epoch', 'main/loss', 'main/accuracy','validation/main/loss', 'validation/main/accuracy', 'elapsed_time' ]))

    print("run")
    trainer.run()

In [10]:
train(train_img, valid_img)

run
epoch       main/loss   main/accuracy  validation/main/loss  validation/main/accuracy  elapsed_time
1           7.35923     0.03           4.11911               0.0070922                 3.26998       
2           4.08771     0.025          3.79103               0.0133333                 6.49509       
3           3.78745     0.03           3.70531               0.0275177                 9.72089       
4           3.70796     0.02           3.67157               0.0133333                 12.9414       
5           3.63572     0.0381818      3.6226                0.0337589                 15.9202       
6           3.63916     0.035          3.63845               0.0070922                 19.1401       
7           3.66247     0.0216667      3.65213               0                         22.3605       
8           3.62315     0.0316667      3.6236                0.0133333                 25.5723       
9           3.60865     0.0327273      3.6305                0.0133333          

79          2.67698     0.17           3.77869               0.0479433                 251.189       
80          2.6517      0.147273       3.35023               0.0675177                 254.168       
81          2.49853     0.203333       3.42873               0.0958865                 257.384       
82          2.32294     0.21           3.15838               0.122553                  260.599       
83          2.14782     0.248333       3.43691               0.142979                  263.815       
84          2.17755     0.245455       3.22306               0.123404                  266.801       
85          2.10614     0.276667       3.28177               0.0746099                 270.016       
86          2.05877     0.265          3.40388               0.149645                  273.233       
87          2.17734     0.251667       3.0731                0.095461                  276.451       
88          2.17024     0.26           3.14248               0.115035             

158         0.330893    0.888333       6.83086               0.191348                  500.667       
159         0.289648    0.903636       6.86298               0.238865                  503.645       
160         0.383985    0.866667       7.93067               0.20383                   506.869       
161         0.384116    0.868333       7.53331               0.219291                  510.078       
162         0.474952    0.848333       6.84931               0.178865                  513.286       
163         0.49407     0.841818       6.99461               0.190496                  516.26        
164         0.34733     0.885          7.72342               0.156738                  519.466       
165         0.268691    0.91           6.88126               0.198014                  522.68        
166         0.158546    0.95           7.92788               0.204681                  525.893       
167         0.203953    0.943636       7.37238               0.211348             

236         0.000667149  1              12.0992               0.191773                  746.824       
237         0.0100845   0.996667       11.9233               0.198865                  750.036       
238         0.0134802   0.996364       11.8006               0.198865                  753.01        
239         0.0273207   0.988333       11.4364               0.225106                  756.223       
240         0.0568425   0.978333       11.5264               0.205532                  759.435       
241         0.117335    0.966667       11.1771               0.176738                  762.669       
242         0.159134    0.949091       9.49268               0.149645                  765.643       
243         0.192251    0.943333       8.42375               0.258865                  768.853       
244         0.337529    0.9            7.95275               0.205106                  772.061       
245         0.407258    0.875          7.14854               0.136312            

314         4.42092e-05  1              13.4781               0.219291                  994.298       
315         6.6665e-05  1              13.5601               0.219291                  997.512       
316         2.89583e-05  1              13.6511               0.219291                  1000.73       
317         3.31792e-05  1              13.7236               0.219291                  1003.72       
318         6.05273e-05  1              13.849                0.219291                  1006.95       
319         3.6087e-05  1              13.9295               0.219291                  1010.17       
320         0.000283194  1              13.5352               0.232624                  1013.38       
321         7.84232e-05  1              13.2669               0.225532                  1016.38       
322         3.05525e-05  1              13.2215               0.232624                  1019.6        
323         0.000126357  1              13.2295               0.225532     

...

4685        4.01686e-06  1              11.1764               0.327234                  14889.6       
4686        4.04211e-05  1              11.1881               0.327234                  14892.9       
4687        1.77101e-05  1              11.1996               0.327234                  14896.2       
4688        3.66789e-06  1              11.2087               0.327234                  14899.2       
4689        3.38336e-06  1              11.2148               0.327234                  14902.5       
4690        5.73675e-06  1              11.2205               0.327234                  14905.7       
4691        3.75787e-06  1              11.2329               0.327234                  14909         
4692        3.75152e-06  1              11.239                0.327234                  14912.1       
4693        5.31912e-06  1              11.2444               0.327234                  14915.3       
4694        3.20315e-06  1              11.2547               0.320567   

4763        0.0371414   0.989091       9.16925               0.346809                  15139.9       
4764        0.0546248   0.988333       8.87637               0.319716                  15143.2       
4765        0.0482307   0.983333       9.18649               0.367234                  15146.5       
4766        0.0540573   0.988333       8.86831               0.354326                  15149.7       
4767        0.120603    0.98           9.1063                0.333901                  15152.7       
4768        0.0970415   0.983333       8.43378               0.285106                  15156         
4769        0.0612075   0.983333       7.83891               0.38766                   15159.3       
4770        0.0616186   0.981667       8.10483               0.367234                  15162.5       
4771        0.121884    0.970909       8.4807                0.36766                   15165.6       
4772        0.198918    0.945          8.40442               0.345532             

4841        4.10599e-05  1              9.42345               0.367234                  15390.3       
4842        6.90387e-05  1              9.43205               0.367234                  15393.3       
4843        0.000144938  1              9.3483                0.360567                  15396.6       
4844        7.89446e-05  1              9.26635               0.360567                  15399.9       
4845        5.18172e-05  1              9.24689               0.360567                  15403.1       
4846        6.3855e-05  1              9.25033               0.360567                  15406.2       
4847        0.000100075  1              9.27756               0.367234                  15409.4       
4848        2.29705e-05  1              9.29969               0.367234                  15412.7       
4849        3.38034e-05  1              9.31497               0.367234                  15415.9       
4850        3.68827e-05  1              9.32113               0.367234    

4919        7.80781e-06  1              10.0068               0.373901                  15640.7       
4920        3.47783e-06  1              10.0161               0.373901                  15644         
4921        7.38692e-05  1              10.1254               0.367234                  15647         
4922        3.558e-06   1              10.1786               0.360567                  15650.3       
4923        7.48595e-06  1              10.2009               0.360567                  15653.5       
4924        5.53846e-06  1              10.2163               0.360567                  15656.8       
4925        2.22022e-06  1              10.2248               0.360567                  15659.8       
4926        5.25117e-06  1              10.2358               0.360567                  15663.1       
4927        1.27202e-05  1              10.2666               0.360567                  15666.4       
4928        6.61191e-05  1              10.4491               0.367234    

4997        0.0417148   0.991667       6.02118               0.36766                   15891.1       
4998        0.0102915   0.998333       6.42415               0.333475                  15894.3       
4999        0.00949544  0.996667       6.19799               0.381418                  15897.6       
5000        0.00411291  0.998182       6.61703               0.368085                  15900.7       
